In [8]:
import glob,os,shutil,time
import numpy as np
from tqdm import tqdm
from molop import molopconfig,AutoParser
from rdkit.Chem.Draw import IPythonConsole
from rdkit import Chem
from rxnb.read import read_xyz
from rxnb.utils import gen_xtb_sh
from rxnb.calc import perform_xtbts_calc
from rxnpredict.descriptors.calc import xTBCalculator,xTBDescriptor,StericDescriptor
from rxnpredict.descriptors.desc import get_xtb_loc_glb_desc,get_steric_loc_desc
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.molSize = 400, 400
molopconfig.quiet()
CWD = os.getcwd()

## Temp Dev
def get_attach_site_idx(rdmol, init_site, dest_at_type=["C"], exclude_idx_lst=[]):
    nei_inf_lst = [[nei.GetSymbol(),nei.GetIdx()] for nei in rdmol.GetAtomWithIdx(init_site).GetNeighbors() \
                    if nei.GetIdx() > init_site and nei.GetIdx() not in exclude_idx_lst and nei.GetSymbol() in dest_at_type]
    assert len(nei_inf_lst) == 1, f"Error: {nei_inf_lst}"
    return nei_inf_lst[0][1]

def read_loc_inf(gjf_file):
    with open(gjf_file,"r") as fr:
        lines = fr.readlines()
    for idx,line in enumerate(lines):
        if '#' in line:
            title_line = idx + 2
            loc_inf = list(map(eval,lines[title_line].strip().split("//")[1].split("  ")[1:]))
            return loc_inf

def read_freeze_inf(gjf_file):
    with open(gjf_file,"r") as fr:
        lines = fr.readlines()
    for idx,line in enumerate(lines):
        if '#' in line:
            title_line = idx + 2
            freeze_inf = list(map(eval,lines[title_line].strip().split("//")[0].split("  ")[1:]))
            return freeze_inf
        
def return_restart_geom(opt_log):
    with open(opt_log,'r') as fr:
        lines = fr.readlines()
    natom = 0
    tag = False
    coords_val_lst = []
    for idx,line in enumerate(lines):
        if natom == 0 and 'NAtoms' in line:
            natom = int(line.strip().split()[1])
        if 'Standard orientation: ' in line:
            coords = np.array([list(map(eval,line.strip().split()[3:3+3])) for line in lines[idx+5:idx+5+natom]])
            tag = True
        if tag and 'Maximum Force' in line:
            if '********' in line:
                max_force = 999999
            else:
                max_force = eval(line.strip().split()[2])
            coords_val_lst.append([coords,max_force])
            tag = False
    if len(coords_val_lst) == 0:
        return None
    coords_val_lst_sorted = sorted(coords_val_lst,key=lambda x: x[1])
    return coords_val_lst_sorted[0]

## Semi-empirical level TS optimization and descriptor calculation

In [6]:
# move gjf file to separate folder
ori_path_lst = sorted(glob.glob('../gen_TS/*.gjf'))


for file in tqdm(ori_path_lst):
    
    filename = os.path.basename(file).split(".")[0]
    dirname = os.path.dirname(file)
    if os.path.exists(f'{dirname}/split/{filename}/{filename}.gjf'):
        continue
    
    if not os.path.exists(f'{dirname}/split/{filename}'):
        os.makedirs(f'{dirname}/split/{filename}')
    shutil.copy(file,f'{dirname}/split/{filename}/{filename}.gjf')
    break # just for demonstration

  0%|          | 0/222 [00:00<?, ?it/s]


The optimized semi-empirical level TS geometry example can be found in `../gen_TS/split/cyc_rp_0/tsopted.xyz`, the xTB calculated descriptor can be found in `../gen_TS/split/cyc_rp_0/xtb/desc_ens.npy`

In [ ]:
src_path = "../gen_TS/split/cyc_rp_0"
               
max_attempt = 5
xtb_Desc = xTBDescriptor()


print(f"[INFO] {src_path} processing...")
os.chdir(src_path)          
if os.path.exists('./4-tsfreq.log') and not AutoParser('./4-tsfreq.log')[0][-1].is_error and os.path.exists(f'{src_path}/xtb/desc.npz'):
    print("[INFO] Task is finished! Skip")
    os.chdir(CWD)

else:
    xyz_file = f"{src_path.split('/')[-1]}.xyz"
    gjf_file = f"{src_path.split('/')[-1]}.gjf"
    freeze_inf = read_freeze_inf(gjf_file)
    mol_obj = AutoParser(gjf_file)[0][0]

    ##  obtain local site index
    loc_inf = read_loc_inf(gjf_file)
    site_idx_lst = []
    bond_idx_lst = []
    attach_idx_lst = []
    for item in loc_inf:
        if isinstance(item, int):
            site_idx_lst.append(item-1)
        elif isinstance(item, tuple):
            if len(item) == 1:
                
                other_attach_idx = get_attach_site_idx(mol_obj.rdmol, item[0]-1, dest_at_type=["C"], exclude_idx_lst=[])
                attach_idx_lst.append([item[0]-1,other_attach_idx])
            else:
                bond_idx_lst.append([item[0]-1,item[1]-1])
        elif isinstance(item, list):
            bond_idx_lst.append([i-1 for i in item])    
    ## 

    chrg,multi = mol_obj.charge,mol_obj.multiplicity
    chrg,multi = 0,1                                
    mol_obj.to_XYZ_file(xyz_file)
    gen_xtb_sh(xtb_sh='./xtb.sh',thread_num=8,gfn=2,xtb_path='xtb')    
    symbols,init_coords = read_xyz(xyz_file)
    if not os.path.exists('./4-tsfreq.log') or AutoParser('./4-tsfreq.log')[0][-1].is_error:
        print("[INFO] Performing optimization...")
        opt_start_time = time.time()
        attempt = 1
        opt_from_step = 1
        while attempt <= max_attempt:
            if attempt == 1:
                finished,opt_log,opt_from_step = perform_xtbts_calc(symbols,init_coords,chrg,multi,
                                                                    fixed_inf_lst=freeze_inf,
                                                                    opt_from_step=opt_from_step,loose=False,
                                                                    )
            else:
                finished,opt_log,opt_from_step = perform_xtbts_calc(symbols,init_coords,chrg,multi,
                                                                    fixed_inf_lst=freeze_inf,
                                                                    opt_from_step=opt_from_step,loose=True)
            if finished:
                break
            elif not finished and opt_log:
                print(f"[INFO] ({attempt}/{max_attempt}) Restarting optimization...")
            else:
                break
            attempt += 1
        
        opt_finish_time = time.time()
        print('+'*10,f'Optimization done, time used: {opt_finish_time - opt_start_time:.2f} s','+'*10)
    else:
        finished = True
        
    if finished:
        desc_start_time = time.time()
        print("[INFO] Calculate descriptor")
        
        xtb_work_dir = './xtb'
        os.makedirs(xtb_work_dir, exist_ok=True)
        shutil.copy('./tsopted.xyz',xtb_work_dir)
        os.chdir(xtb_work_dir)
        xtb_calc = xTBCalculator('.',"tsopted.xyz",chrg=0,gfn=2)
        xtb_calc_flag = xtb_calc.single_point()
        xtb_desc = xtb_Desc.read_desc(xtb_dir='.',xyz_file="tsopted.xyz")
        
        xtb_desc_values,xtb_desc_names = get_xtb_loc_glb_desc(xtb_desc['global'],xtb_desc['local'],
                                                                site_idx_lst,bond_idx_lst)
        steric_loc_desc_values,steric_loc_desc_names = get_steric_loc_desc(f'tsopted.xyz',
                                                                    idx_for_bv=[site_idx_lst]+attach_idx_lst,
                                                                    idx_for_sterimol=attach_idx_lst)
        tot_desc_values = np.concatenate([xtb_desc_values,steric_loc_desc_values],axis=0)
        tot_desc_names = xtb_desc_names + steric_loc_desc_names
        print("[INFO] Descriptors saving")
        np.savez(f'./desc.npz',
        desc_values=tot_desc_values,
        desc_names=tot_desc_names)
        desc_end_time = time.time()
        print('+'*10,f'Descriptor done, time used: {desc_end_time - desc_start_time:.2f} s','+'*10)
        pass
    os.chdir(CWD)
        

[INFO] ../gen_TS/split/cyc_rp_0 processing...
[INFO] Performing optimization...
[INFO] (1/4) Start performing constrained optimization...
[INFO] (1/4) Constrained optimization finished
[INFO] (2/4) Start performing frequency calculation...
[INFO] (2/4) Frequency calculation finished
[INFO] The first frequency is -801.5657958984375 cm-1, the number of imaginary frequencies is 1
[INFO] (3/4) Start performing TS optimization...
[INFO] (3/4) TS optimization finished
[INFO] (4/4) Start performing frequency calculation...
[INFO] (4/4) Frequency calculation finished
[INFO] The first frequency is -623.5753173828125 cm-1, the number of imaginary frequencies is 1
++++++++++ Optimization done, time used: 120.16 s ++++++++++
[INFO] Calculate descriptor
[INFO] Single point calculation for tsopted.xyz
[INFO] Descriptors saving
++++++++++ Descriptor done, time used: 2.77 s ++++++++++
